In [1]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [2]:
!kaggle datasets download -d gauravduttakiit/fakereal-logo-detection

100% 27.0M/27.0M [00:01<00:00, 29.2MB/s]
100% 27.0M/27.0M [00:01<00:00, 19.8MB/s]


In [3]:
from zipfile import ZipFile
dataset = '/content/fakereal-logo-detection.zip'

with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print('The dataset is extracted')

The dataset is extracted


In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2
from google.colab.patches import cv2_imshow
from PIL import Image

In [5]:
train_folder=os.listdir('/content/train')
print(train_folder)
print(len(train_folder))

['Genuine', 'Fake']
2


In [8]:
data=[]

In [7]:
fake_logo_folder=os.listdir('/content/train/Fake')
print(fake_logo_folder)
print(len(fake_logo_folder))

['scal_000005_fdf3ec20da3441c0b4f78087cc7910f9.jpg', 'scal_000005_3f2e50e35c6341d980a36b383fac5652.jpg', 'scal_000003_dc31b086350f4902aaac14da9646d0f3.jpg', '000004_e23ac0a097d745cf9088b4afe9a52766.jpg', 'scal_000001_731c80275bd64210b6bd64bfe6eb9a8f.jpg', 'scal_000002_3611d6d9643247178e123b92c26559d6.jpg', '000005_5c7317d3e3474b0b9d56edaf324519ac.jpg', 'scal_000005_60eabe5b53df4f9bb6fadbe8c9b85a52.jpg', 'scal_000002_dfa777556b4644899fcc03fb21947acb.jpg', 'scal_000005_bc7f245635a44d088dfc8ac0c1898dcd.jpg', '000002_4503b9481ee14f309c41ad6b6e940897.jpg', 'scal_000004_64428b25019c4caca6bcc6e452e1934d.jpg', 'scal_000003_24f7603d37cf44f28fb89a387b6fd68a.jpg', 'scal_000003_1d160dd8b1614c45b530c8f74f599b78.jpg', 'scal_000005_7b15e32aea5f4aeeb43be2d1636a85dd.jpg', 'scal_000002_b7da65a1d41644c6ba715811e32aa442.jpg', 'scal_000003_494d5d60fce54a7b9c6bcf96aa8a2cb4.jpg', '000005_e477f897f6a246ed8cb5a0327939072f.jpg', '000001_9ab93958c5244fb38d4a41a4a84344ef.jpg', '000003_79c36f72c40f498e82468cd400a5

In [11]:
for i in range(439):
  img_path='/content/train/Fake/'+fake_logo_folder[i]
  img_pillow=Image.open(img_path)
  img_color=img_pillow.convert('RGB')
  img_resize=img_color.resize((64,64))
  img_arr=np.array(img_resize)
  data.append([img_arr,1])

In [12]:
genuine_logo_folder=os.listdir('/content/train/Genuine')
print(genuine_logo_folder)
print(len(genuine_logo_folder))

['000005_b72b53b9100a4d2a89beaf431caf08c5.jpg', '000003_56980ae8fbe6493aa1d2baf201a5ba67.jpg', '000004_27faebf251d24522a19e5b4412b1cd73.jpg', '000005_27f396005a7f4e7d89a3c381f98328ca.jpg', '000001_ee1c817ab6374d7eb855e0b24fce7bfd.jpg', '000002_6d5b09bf75124f7ca31d2df937672663.jpg', '000003_3f5cfd980562453ab99f3e16250fc752.jpg', '000004_82d0437f56094b409c0e543d0c918d4c.jpg', '000005_cfeef2df9fff4fc6ae2acf0e77798ef3.jpg', '000002_8f62883b16824f22bde16d585240f61b.jpg', '000003_4b4d618c6d76415cb69ad5a2a258db5f.jpg', '000005_7577aad96d5e4703b969a25a0d5906b9.jpg', '000001_132612d8fb7f42aba0b4f370908123d5.jpg', '000002_210dd0eac9dc4c7299442e58a03b7baf.jpg', '000001_2f924414e9af43e790e0c727418833d5.jpg', '000002_524eb369e1fb43108e67798301e12531.jpg', '000001_51a7227ce74f4aaf87ca9699e902bd91.jpg', '000002_14d1fd241ea946c98895901bf2b08cee.jpg', '000004_7e9edf45e65b41f8a68fbe3b6bb750b5.jpg', '000003_fe0a6bdb80a04afc8d737d106353eddd.jpg', '000003_0ef19501d88d47f8a65e60e32e50a612.jpg', '000005_6220

In [13]:
for i in range(220):
  img_path='/content/train/Genuine/'+genuine_logo_folder[i]
  img_pillow=Image.open(img_path)
  img_color=img_pillow.convert('RGB')
  img_resize=img_color.resize((64,64))
  img_arr=np.array(img_resize)
  data.append([img_arr,0])

In [14]:
x=[]
y=[]
for img,label in data:
  x.append(img)
  y.append(label)

In [15]:
x=np.array(x)
y=np.array(y)

In [16]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.1,random_state=0)

In [17]:
x_train_scaled=x_train/255

In [18]:
import tensorflow as tf
from tensorflow import keras

In [19]:
num_of_classes=2
model=keras.Sequential()

model.add(keras.layers.Conv2D(32,kernel_size=(3,3),activation='relu',input_shape=(64,64,3)))
model.add(keras.layers.MaxPooling2D(pool_size=(2,2)))

model.add(keras.layers.Conv2D(64,kernel_size=(3,3),activation='relu'))
model.add(keras.layers.MaxPooling2D(pool_size=(2,2)))

model.add(keras.layers.Flatten())

model.add(keras.layers.Dense(128,activation='relu'))
model.add(keras.layers.Dropout(0.5))

model.add(keras.layers.Dense(64,activation='relu'))
model.add(keras.layers.Dropout(0.5))

model.add(keras.layers.Dense(num_of_classes,activation='softmax'))

In [20]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [22]:
model.fit(x_train_scaled,y_train,epochs=50,validation_split=0.1)

Epoch 1/50
17/17 [==============================] - 6s 360ms/step - loss: 0.4661 - accuracy: 0.7280 - val_loss: 0.5875 - val_accuracy: 0.6500
Epoch 2/50
17/17 [==============================] - 3s 201ms/step - loss: 0.4624 - accuracy: 0.7054 - val_loss: 0.6304 - val_accuracy: 0.6833
Epoch 3/50
17/17 [==============================] - 3s 171ms/step - loss: 0.4602 - accuracy: 0.7167 - val_loss: 0.6349 - val_accuracy: 0.5500
Epoch 4/50
17/17 [==============================] - 3s 175ms/step - loss: 0.4491 - accuracy: 0.7636 - val_loss: 0.6530 - val_accuracy: 0.6167
Epoch 5/50
17/17 [==============================] - 4s 214ms/step - loss: 0.4229 - accuracy: 0.7467 - val_loss: 0.7207 - val_accuracy: 0.5667
Epoch 6/50
17/17 [==============================] - 4s 209ms/step - loss: 0.4446 - accuracy: 0.7336 - val_loss: 0.6563 - val_accuracy: 0.6000
Epoch 7/50
17/17 [==============================] - 3s 172ms/step - loss: 0.4456 - accuracy: 0.7355 - val_loss: 0.7565 - val_accuracy: 0.5500
Epoch 

In [25]:
img_path='/content/train/Genuine/000001_07cbc019bcf34352bf73e821ae50340a.jpg'
img_pillow=Image.open(img_path)
img_color=img_pillow.convert('RGB')
img_resize=img_color.resize((64,64))
img_arr=np.array(img_resize)
img_scaled=img_arr/255
img_reshaped=np.reshape(img_scaled,[1,64,64,3])
prediction=model.predict(img_reshaped)
print(prediction)
output=np.argmax(prediction)
if(output==0):
  print('The LOGO is GENUINE')
else:
  print('The LOGO is FAKE')

1/1 [==============================] - 0s 37ms/step
[[1.0000000e+00 1.2298834e-10]]
The LOGO is GENUINE
